# LeNet

Among the first ground breaking result for CNN

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In the 90s, optical character recognition was a hot topic  
Especially for banks who had to had to process millions of bank check a year (at that time, credit cards were not as popular as today)

In 1998, Yann LeCun, Leon Bottou, Yoshua Bengio, and Patrick Haffner proposed a CNN architecture and a gradient-based approach to train it on handwritten character recognition (**MNIST** dataset)

<center>
    <img src='images/mnist.png' width="65%" style="margin-left:auto; margin-right:auto"/>
    <p style="font-size:14px;">Source: <a href='http://vision.stanford.edu/cs598_spring07/papers/Lecun98.pdf'>LeCun's MNIST paper</a></p>
</center>

In [2]:
class LeCunRevisited(nn.Module):
    
    def __init__(self):
        super(LeCunRevisited, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.max_pool = nn.MaxPool2d(kernel_size=2) # original uses AvgPool, kernel size 2 divide the ouput by 4
        
    def forward(self, x):
        x = self.max_pool(F.relu(self.conv1(x)))
        x = self.max_pool(F.relu(self.conv2(x)))
        x = x.reshape(x.size(0), -1)
        x = F.relu(self.fc1(x)) # original uses Sigmoid
        x = F.relu(self.fc2(x))
        return self.fc3(x)

In [4]:
net = LeCunRevisited()
net(torch.randn(2, 1, 32, 32))

tensor([[ 0.0720, -0.0258,  0.0060,  0.0449, -0.0777,  0.1418,  0.0007, -0.0602,
         -0.2036, -0.1169],
        [ 0.0862, -0.0037,  0.0119,  0.0524, -0.0584,  0.1584,  0.0135, -0.0400,
         -0.2225, -0.0981]], grad_fn=<AddmmBackward>)